In [10]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local[4]").getOrCreate()

# get context from the session
sc = spark.sparkContext

In [11]:
Afile = 'A3200x1600.txt'
Bfile = 'B1600x160.txt'
Asize = [3200, 1600]
Bsize = [1600, 160]

In [12]:
rdd1 = sc.textFile(Afile).union(sc.textFile(Bfile)).map(lambda t: tuple(t.split(',')))

In [13]:
def matmul_map(t):
    name, i, j, v = t
    if name == "A":
        for k in range(Bsize[1]):
            yield ((i, k), (j, v))
    else:
        for k in range(Asize[0]):
            yield ((k, j), (i, v))

In [14]:
def matmul_red(it):
    tmp = np.ones(Asize[1])
    for i, v in it:
        tmp[i] *= v
    return sum(tmp)

In [ ]:
rdd1.flatMap(matmul_map).groupByKey().mapValues(matmul_red).map(lambda t: t[1]).reduce(lambda a,b:a+b)

In [5]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [7]:
spark.conf.set("spark.sql.shuffle.partitions", 16)

In [8]:
spark.conf.set("spark.worker.cleanup.enabled", True)

In [9]:
spark.stop()